In [1]:
import argparse 
import os
from pathlib import Path
import torch
import numpy as np
import cv2
from models.common import DetectMultiBackend

In [2]:
def resize_with_padding(image, target_size=1024):
    h, w, _ = image.shape
    ratio = target_size / max(h, w)
    new_h, new_w = int(h * ratio), int(w * ratio)
    image_resized = cv2.resize(image, (new_w, new_h))
    
    top_padding = (target_size - new_h) // 2
    bottom_padding = target_size - new_h - top_padding
    left_padding = (target_size - new_w) // 2
    right_padding = target_size - new_w - left_padding

    padded_image = cv2.copyMakeBorder(image_resized, top_padding, bottom_padding, 
                                      left_padding, right_padding, cv2.BORDER_CONSTANT, value=(0, 0, 0))
    return padded_image

In [4]:
def load_image(img_path, device):
    image = cv2.imread(img_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image_resized = resize_with_padding(image, target_size=1024)
    img_tensor = torch.from_numpy(image_resized).float().permute(2, 0, 1).unsqueeze(0) / 255.0
    img_tensor = img_tensor.to(device)

    return img_tensor

In [3]:
def load_model(weights, device, out_channels=1):
    model = DetectMultiBackend(weights, device=device)
    model.eval()
    return model

In [5]:
weights = 'runs/train/BMblood/CP-yolov9-c/weights/best.pt'
img_path = '../data/BMBlood/IN/val/images/987.jpg'
save_path = 'runs/heatmap/987_heatmap.png'
device = 'cuda:1'

In [7]:
device = torch.device(device)
model = load_model(weights, device)
img = load_image(img_path, device)

Fusing layers... 
yolov9-c summary: 604 layers, 50790678 parameters, 0 gradients, 237.1 GFLOPs


In [ ]:
# 查看model.38的子模型结构，使用 named_modules() 打印层名称和路径

'''
model.38.cv2是一个container，是ModuleList，不适合作为钩子
'''

print("\nSubmodules under model.38:")
for name, module in model.model.named_modules():
    if name.startswith("model.38"):
        print(f"{name}: {type(module)}")


Submodules under model.38:
model.38: <class 'models.yolo.DualDDetect'>
model.38.cv2: <class 'torch.nn.modules.container.ModuleList'>
model.38.cv2.0: <class 'torch.nn.modules.container.Sequential'>
model.38.cv2.0.0: <class 'models.common.Conv'>
model.38.cv2.0.0.conv: <class 'torch.nn.modules.conv.Conv2d'>
model.38.cv2.0.1: <class 'models.common.Conv'>
model.38.cv2.0.1.conv: <class 'torch.nn.modules.conv.Conv2d'>
model.38.cv2.0.2: <class 'torch.nn.modules.conv.Conv2d'>
model.38.cv2.1: <class 'torch.nn.modules.container.Sequential'>
model.38.cv2.1.0: <class 'models.common.Conv'>
model.38.cv2.1.0.conv: <class 'torch.nn.modules.conv.Conv2d'>
model.38.cv2.1.1: <class 'models.common.Conv'>
model.38.cv2.1.1.conv: <class 'torch.nn.modules.conv.Conv2d'>
model.38.cv2.1.2: <class 'torch.nn.modules.conv.Conv2d'>
model.38.cv2.2: <class 'torch.nn.modules.container.Sequential'>
model.38.cv2.2.0: <class 'models.common.Conv'>
model.38.cv2.2.0.conv: <class 'torch.nn.modules.conv.Conv2d'>
model.38.cv2.2.

In [9]:
print(model.model)

DetectionModel(
  (model): Sequential(
    (0): Silence()
    (1): Conv(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (2): Conv(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1))
      (act): SiLU(inplace=True)
    )
    (3): RepNCSPELAN4(
      (cv1): Conv(
        (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1))
        (act): SiLU(inplace=True)
      )
      (cv2): Sequential(
        (0): RepNCSP(
          (cv1): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv2): Conv(
            (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (cv3): Conv(
            (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
            (act): SiLU(inplace=True)
          )
          (m): Sequential(
            (0): RepN

 查看22层主干网络的结构

In [1]:
import torch
from models.yolo import Model

In [2]:
device = torch.device("cuda:1")
cfg = "./data/gelan-c.yaml"
model = Model(cfg, ch=3, nc=41, anchors=3)
#model = model.half()
model = model.to(device)
_ = model.eval()
ckpt = torch.load('runs/train/BMblood/CP-yolov9-c/weights/best.pt', map_location='cuda:1')
model.names = ckpt['model'].names
model.nc = ckpt['model'].nc

Overriding model.yaml anchors with anchors=3

                 from  n    params  module                                  arguments                     
  0                -1  1      1856  models.common.Conv                      [3, 64, 3, 2]                 
  1                -1  1     73984  models.common.Conv                      [64, 128, 3, 2]               
  2                -1  1    212864  models.common.RepNCSPELAN4              [128, 256, 128, 64, 1]        
  3                -1  1    164352  models.common.ADown                     [256, 256]                    
  4                -1  1    847616  models.common.RepNCSPELAN4              [256, 512, 256, 128, 1]       
  5                -1  1    656384  models.common.ADown                     [512, 512]                    
  6                -1  1   2857472  models.common.RepNCSPELAN4              [512, 512, 512, 256, 1]       
  7                -1  1    656384  models.common.ADown                     [512, 512]            

In [3]:
print(model.model)

Sequential(
  (0): Conv(
    (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
  (1): Conv(
    (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
    (act): SiLU(inplace=True)
  )
  (2): RepNCSPELAN4(
    (cv1): Conv(
      (conv): Conv2d(128, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn): BatchNorm2d(128, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
      (act): SiLU(inplace=True)
    )
    (cv2): Sequential(
      (0): RepNCSP(
        (cv1): Conv(
          (conv): Conv2d(64, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
      

In [11]:
print("\nSubmodules under model.22:")
for name, module in model.named_modules():
    if name.startswith("model.22"):
        print(f"{name}: {type(module)}")


Submodules under model.22:
model.22: <class 'models.yolo.DDetect'>
model.22.cv2: <class 'torch.nn.modules.container.ModuleList'>
model.22.cv2.0: <class 'torch.nn.modules.container.Sequential'>
model.22.cv2.0.0: <class 'models.common.Conv'>
model.22.cv2.0.0.conv: <class 'torch.nn.modules.conv.Conv2d'>
model.22.cv2.0.0.bn: <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
model.22.cv2.0.1: <class 'models.common.Conv'>
model.22.cv2.0.1.conv: <class 'torch.nn.modules.conv.Conv2d'>
model.22.cv2.0.1.bn: <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
model.22.cv2.0.2: <class 'torch.nn.modules.conv.Conv2d'>
model.22.cv2.1: <class 'torch.nn.modules.container.Sequential'>
model.22.cv2.1.0: <class 'models.common.Conv'>
model.22.cv2.1.0.conv: <class 'torch.nn.modules.conv.Conv2d'>
model.22.cv2.1.0.bn: <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
model.22.cv2.1.1: <class 'models.common.Conv'>
model.22.cv2.1.1.conv: <class 'torch.nn.modules.conv.Conv2d'>
model.22.cv2.1.1.bn: <class 'torch.nn.m

In [8]:
print("\nSubmodules under model:")
for name, module in model.model.named_modules():
    print(f"{name}: {type(module)}")


Submodules under model:
: <class 'torch.nn.modules.container.Sequential'>
0: <class 'models.common.Conv'>
0.conv: <class 'torch.nn.modules.conv.Conv2d'>
0.bn: <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
0.act: <class 'torch.nn.modules.activation.SiLU'>
1: <class 'models.common.Conv'>
1.conv: <class 'torch.nn.modules.conv.Conv2d'>
1.bn: <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
2: <class 'models.common.RepNCSPELAN4'>
2.cv1: <class 'models.common.Conv'>
2.cv1.conv: <class 'torch.nn.modules.conv.Conv2d'>
2.cv1.bn: <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
2.cv2: <class 'torch.nn.modules.container.Sequential'>
2.cv2.0: <class 'models.common.RepNCSP'>
2.cv2.0.cv1: <class 'models.common.Conv'>
2.cv2.0.cv1.conv: <class 'torch.nn.modules.conv.Conv2d'>
2.cv2.0.cv1.bn: <class 'torch.nn.modules.batchnorm.BatchNorm2d'>
2.cv2.0.cv2: <class 'models.common.Conv'>
2.cv2.0.cv2.conv: <class 'torch.nn.modules.conv.Conv2d'>
2.cv2.0.cv2.bn: <class 'torch.nn.modules.batchnorm.BatchNorm2d